# Code to generate Frustration Fouls and Euro Fouls

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/frustration_coast/"

In [ ]:
league = "NBA"
season_yr = "2019"
season_type = "Regular Season"
games_id = pbp_season(league=league,season_yr=season_yr,season_type=season_type)
games_list = pbp_games(games_id)

In [ ]:
year = int(season_yr)
# player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
player_dict = get_players_pbp(league = league)
team_dict = teams.get_teams() # Creating Team Dictionary

In [ ]:
from pbpstats.resources.enhanced_pbp import Foul
from pbpstats.resources.enhanced_pbp import Turnover
from pbpstats.resources.enhanced_pbp import FieldGoal
pos_store = []
TO_Miss_pID = []
Foul_pID = []
Foul_tID = []
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, Foul) and (isinstance(possession_event.previous_event, Turnover) or  (isinstance(possession_event.previous_event, FieldGoal) and not possession_event.previous_event.is_made)) and possession_event.seconds_since_previous_event <= 5:
                pos_store.append(possession)
                TO_Miss_pID.append(possession_event.previous_event.player1_id)
                # print ("Turnover/Missed Shot Player: {0}".format(possession_event.previous_event.player1_id))
                Foul_pID.append(possession_event.player1_id)
                Foul_tID.append(possession_event.team_id)
                # print ("Foul Player: {0}".format(possession_event.player1_id))

In [ ]:
Foul_pID1, Fouls = np.unique(Foul_pID, return_counts=True)  
Player_Name = np.array([player_dict.get(x,np.nan) for x in Foul_pID1])

In [ ]:
data_euroP = pd.DataFrame({'Foul_pID':Foul_pID1,'Player_Name':Player_Name,'Fouls':Fouls})
data_euroP = data_euroP[data_euroP['Player_Name'] != np.nan]
# data_euroP.to_csv ('Fast_Fouls_20_21_updated.csv',index=False)

In [ ]:
Foul_tID1, Fouls = np.unique(Foul_tID, return_counts=True)  
Team_Name = [team['full_name'] for team, tID in zip(team_dict,Foul_tID1) if team['id'] ==tID]

In [ ]:
np.sum(Fouls)

In [ ]:
data_euroT = pd.DataFrame({'Foul_tID':Foul_tID1,'Team_Name':Team_Name,'Fouls':Fouls})
# data_euroT.to_csv ('Fast_Fouls_20_21_teams_updated.csv',index=False)

In [ ]:
FFoul_pID =[]
k = 0
for i in range(len(TO_Miss_pID)):
    if TO_Miss_pID[i] == Foul_pID[i]:
        FFoul_pID.append(Foul_pID[i])
        k +=1
        print(k)
        clear_output(wait=True)

In [ ]:
FFoul_pID1, Fouls = np.unique(FFoul_pID, return_counts=True)  
Player_Name = np.array([player_dict.get(x,np.nan) for x in FFoul_pID1])

In [ ]:
data_frust = pd.DataFrame({'FFoul_pID':FFoul_pID1,'Player':Player_Name,'Fouls':Fouls})
data_frust = data_frust[data_frust['Player'] != np.nan]
# data_frust.to_csv ('Frustration_Fouls_20_21_updated.csv',index=False)

In [ ]:
data_frust['rank'] = data_frust['Fouls'].rank(ascending=False) 
data_frust = data_frust.sort_values(by=['rank'])
data_frust_plot = data_frust.drop(columns=['FFoul_pID', 'rank'])
data_frust_plot.reset_index(drop=True,inplace=True)
data_frust_plot.reset_index(drop=False,inplace=True)
data_frust_plot["#"] = data_frust_plot["index"] +1
data_frust_plot = data_frust_plot[["#","Player","Fouls"]]
data_frust_plot = data_frust_plot.head(10)

In [ ]:
def plot_table(df,var,sort="Player",title=" ",col_width=15):
    fig = go.Figure(data=[go.Table(
        columnwidth=[5,40,col_width],
        header=dict(values=list(df.columns),
                    fill_color='blue',
                    align=['center','left','center'],
                    font=dict(color='snow',family="Arial Black, monospace", size=12),
                    line_color="snow"
                    ),
        cells=dict(values=[df["#"],df[sort],df[var]],
                fill_color='lavender',
                align=['center','left','center'],
                height=23,
                line_color="grey",#lightgrey",
                ),
                # height=25
        ),
    ])
    # fig.update_layout(title_text=title)
    fig.update_layout(title=dict(text=title,y=0.98,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
    tab_width = 250 + col_width
    tab_height = 310
    fig.update_layout(width=tab_width,height=tab_height,margin=dict(t=25,b=1,l=1,r=1))
    fig.add_annotation(x=0.0, y=0.0,text="@SravanNBA",showarrow=False,xshift=1,yshift=5)
    fig.add_annotation(x=1.0, y=0.0,text="Source: pbpstats",showarrow=False,xshift=1,yshift=5,font=dict(size=10))
    # fig.update_layout(autosize=True)
    fig.show()
    # fig.write_image(fig_DIR+f"{var}.png", scale=2)
    return fig

In [ ]:
if league == "NBA":
    title = f"Frustration Fouls {year}-{year+1}"
elif league == "WNBA":
    title = f"Frustration Fouls {year}"
fig = plot_table(data_frust_plot,"Fouls",title=title,col_width=15)
fig.write_image(fig_DIR+f"{league}_frust_foul_{year}.png", scale=3)

In [ ]:
Foul_tID1, Fouls = np.unique(Foul_tID, return_counts=True)  
Team_Name = [team['full_name'] for team, tID in zip(team_dict,Foul_tID1) if team['id'] ==tID]

data_euro = pd.DataFrame({'Foul_tID':Foul_tID1,'Team':Team_Name,'Fouls':Fouls})
data_euro['rank'] = data_euro['Fouls'].rank(ascending=False) 
data_euro = data_euro.sort_values(by=['rank'])
data_euro_plot = data_euro.drop(columns=['Foul_tID', 'rank'])
data_euro_plot.reset_index(drop=True,inplace=True)
data_euro_plot.reset_index(drop=False,inplace=True)
data_euro_plot["#"] = data_euro_plot["index"] +1
data_euro_plot = data_euro_plot[["#","Team","Fouls"]]
data_euro_plot = data_euro_plot.head(10)

In [ ]:
if league == "NBA":
    title = f"Euro Fouls {year}-{year+1}"
elif league == "WNBA":
    title = f"Euro Fouls {year}"
fig = plot_table(data_euro_plot,"Fouls",sort="Team",title=title,col_width=15)
fig.write_image(fig_DIR+f"{league}_euro_foul_{year}.png", scale=3)